In [1]:
import requests
from bs4 import BeautifulSoup
import matplotlib
import matplotlib.pyplot as plt
import pandas

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
def PlayerStatsFetch():
    
    url = "https://www.pro-football-reference.com/years/2024/scoring.htm"
    response = requests.get(url)

    if response.status_code == 200:
        page_content = response.text
        soup = BeautifulSoup(str(page_content), 'html.parser')
        tables = soup.find_all("table")   
        data=[]
        if tables:
            for table in tables:
                
                rows = table.find_all("tr") 
                for row in rows:
                    row_data = [cell.get_text() for cell in row.find_all(["th", "td"])]
                    data.append(row_data)
    else:
        print("Failed to retrieve the webpage.")
    return data
    

In [13]:
teams_data = PlayerStatsFetch()[1:]
teams = {}
for r in teams_data:
    if r[2] in teams:
        teams[r[2]].append([r[1],r[7],r[8],r[14]])
    else:
        teams[r[2]] = [[r[1],r[7],r[8],r[14]]]
teams_data = teams



In [4]:
def PlayByPlayFetch(token):
    
    url = f"https://www.basketball-reference.com/boxscores/pbp/{token}.html"
    response = requests.get(url)
    if response.status_code == 200:
        page_content = response.text
        soup = BeautifulSoup(str(page_content), 'html.parser')
        tables = soup.find_all("table")   
        
        if tables:
            for table in tables:
                data={}
                rows = table.find_all("tr") 
                for row in rows:
                    row_data = [cell.get_text() for cell in row.find_all(["th", "td"])]
                    if len(row_data) == 1:
                        title = row_data[0]
                        data[title] = []
                    data[title].append(row_data)
    return data
play_data = PlayByPlayFetch("202310240DEN")


In [5]:
player_mod = {}
for player in Lakers_roster:
    temp = player.split(" ")
    player_mod[temp[0][0]+". "+temp[1]] = []

player_mod.keys()
for q in play_data:
    for pd in play_data[q]:
        for pl in player_mod.keys():
            if len(pd)>1 and (pl in pd[1]):
                player_mod[pl].append([q]+pd)

In [6]:
for temp in player_mod["A. Davis"]:
    print(temp)

['1st Q', '12:00.0', 'Jump ball: A. Davis vs. N. JokiÄ\x87 (L. James gains possession)']
['1st Q', '11:42.0', 'A. Davis makes 2-pt dunk from 1 ft (assist by D. Russell)', '+2', '2-0', '\xa0', '\xa0']
['1st Q', '9:49.0', 'A. Davis misses 2-pt jump shot from 18 ft', '\xa0', '8-7', '\xa0', '\xa0']
['1st Q', '7:44.0', 'Shooting foul by N. JokiÄ\x87 (drawn by A. Davis)', '\xa0', '10-16', '\xa0', '\xa0']
['1st Q', '7:44.0', 'A. Davis makes free throw 1 of 2', '+1', '11-16', '\xa0', '\xa0']
['1st Q', '7:44.0', 'A. Davis makes free throw 2 of 2', '+1', '12-16', '\xa0', '\xa0']
['1st Q', '7:02.0', 'A. Davis makes 2-pt layup from 3 ft', '+2', '14-18', '\xa0', '\xa0']
['1st Q', '6:15.0', 'A. Davis misses 2-pt layup from 6 ft', '\xa0', '16-20', '\xa0', '\xa0']
['1st Q', '5:33.0', 'A. Davis misses 2-pt layup from 5 ft', '\xa0', '16-20', '\xa0', '\xa0']
['1st Q', '5:03.0', 'Turnover by A. Davis (bad pass; steal by A. Gordon)', '\xa0', '16-22', '\xa0', '\xa0']
['1st Q', '4:25.0', 'Turnover by A. Davi

In [7]:
player_mod = {}
for player in Lakers_roster:
    temp = player.split(" ")
    player_mod[temp[0][0]+". "+temp[1]] = []

player_mod.keys()
for q in play_data:
    for pd in play_data[q]:
        for pl in player_mod.keys():
            if any(pl in x for x in pd):
                player_mod[pl].append([q]+pd)
                

In [17]:
df = pandas.DataFrame(player_mod["A. Reaves"], columns=['Quarter', 'Time', 'Play1','PTS','Score','Play2','Play3'])
df

,Quarter,Time,Play1,PTS,Score,Play2,Play3
0,1st Q,10:01.0,Offensive rebound by A. Reaves,,8-7,,
1,1st Q,9:14.0,Defensive rebound by A. Reaves,,10-9,,
2,1st Q,8:39.0,A. Reaves misses 2-pt jump shot from 11 ft,,10-11,,
3,1st Q,5:39.0,,,16-20,,Turnover by K. Caldwell-Pope (lost ball; steal...
4,1st Q,3:26.0,A. Reaves makes 2-pt layup from 3 ft,+2,18-27,,
5,1st Q,2:56.0,A. Reaves misses 2-pt jump shot from 12 ft,,18-29,,
6,1st Q,2:50.0,D. Russell enters the game for A. Reaves,,18-29,,
7,2nd Q,7:05.0,A. Reaves enters the game for D. Russell,,32-44,,
8,2nd Q,5:31.0,A. Reaves makes 2-pt jump shot from 20 ft,+2,38-52,,
9,2nd Q,5:02.0,A. Reaves misses 2-pt jump shot from 20 ft,,38-54,,
